# Deploy mlflow en un cloud Run

### local
Mlflow puede funcionar de forma local de forma simple: se crea una carpeta donde se van a almacenar los resultados, luego al correr mlflow se define que dicha carpeta será la conexión con mlflow. Luego, si se desea ver los resultados en una interfaz web basta con que en la consola se corra "mlflow server" para correr en un local host el servidor de mlflow


### cloud: cluster  + cloud sql
Una forma de habilitar mlflow en cloud es tener un cluster kubernetes encendido y una instancia de cloud sql. Esto gasta dinero al estar siempre encendido y es innecesario si se desea utilizar mlflow solo como una herramienta para registrar los resultados de experimentación de modelos


### cloud: GCS + cloud run
Esta es la solución más sencilla. En local se guardaba los resultados en una carpeta, en cloud se van a guardar en google cloud storage. Luego para mostrar la interfaz web se crea un servicio de cloud run que al abrirlo se muestra la UI de mlflow y solo gasta recursos mientras está abierto. En este notebook se va a mostrar cómo hacer esta implementación

### INFO MLFLOW

Documentación cómo son guardados los resultados: https://mlflow.org/docs/latest/tracking.html#how-runs-and-artifacts-are-recorded

Al correr el código para crear el servidor:

`mlflow server`

En resumen, se deben definir 2 variables:
- `--backend-store-uri` La que se utiliza para especificar la ubicación del almacenamiento de backend utilizado para almacenar y recuperar los datos de seguimiento de MLflow, como métricas, parámetros, artefactos y registros de ejecución. (Por ejemplo: file:///path/to/mlruns)

- `--default-artifact-root` La que debe indicar la ubicación de los artefactos (La data que es más pesada, en comparación con métricas, parámetros, registros de experimentos)


Un ejemplo de implementación es:

`mlflow server \
    --backend-store-uri /mnt/persistent-disk \
    --default-artifact-root s3://my-mlflow-bucket/ \
    --host 0.0.0.0`



Los recursos permitidos son:
- `--backend-store-uri`: local // MLflow supports the database dialects mysql, mssql, sqlite, and postgresql

- `--default-artifact-root`: local // Servicios de Storage de las diferentes nubes


**Consideraciones para implementación cloud**
- Se puede crear un servidor de mlflow alojado en un cloud run pero se debe de tener cuidado que si alguna de estas variables se guardan en local, va a ser el local del cloud run y cuando la instancia se desactive, toda la info va a desaparecer

- Additionally, you should ensure that the --backend-store-uri (which defaults to the ./mlruns directory) points to a persistent (non-ephemeral) disk or database connection. (Sources: https://mlflow.org/docs/latest/tracking.html#networking)

In [ ]:
from google.cloud import storage
import yaml

### 0. Parámetros

In [ ]:
# generales GCP
PROJECT_ID = '{project_gcp}'
REGION = 'us-east1'

# bucket para mlflow
BUCKET_MLFLOW_CLOUDRUN = '{bucket_mlflow}'

# repo de artifact registry
NAME_REPO = 'repo-mlflow-cloudrun-2-2-1'
FORMAT_REPO = 'docker'
DESCRIPTION_REPO = "repo para mlflow serverless 2.2.1"
NAME_IMAGE = 'mlflow-cloudrun-2-2-1'

# nombre de cloud run
NAME_CLOUD_RUN = 'mlflow-cloudrun'

### 1. Setear proyecto GCP

In [ ]:
! gcloud config set project $PROJECT_ID

### 2. Crear bucket

In [ ]:
def create_bucket(project, bucket_name, region):
    '''
    Crear el bucket en GCS
    '''
    storage_client = storage.Client(project = project)
    bucket = storage_client.create_bucket(bucket_name, location = region)
    print("Bucket {} created".format(bucket.name))

In [ ]:
# bucket GCS para mlflow
create_bucket(project = PROJECT_ID, 
              bucket_name = BUCKET_MLFLOW_CLOUDRUN, 
              region = REGION
             )

### 2.B Crear carpetas en GCS
- "mlflow_artifacts" Para almacenar los artefactos
- "mlflow_backend" Para almacenar el backend (registros experimentos, métricas, parámetros, etc)

In [ ]:
# actualmente lo hago manual

### 3. Obtener SA que tenga los permisos para leer y guardar en GCS

In [ ]:
# json con credenciales - service account
path_credentials = 'sa-gcp-mlflow.json'

### 4. Crear dockerfile para cloud run

En python se puede crear un string que represente al dockerfile (pudiendo modificarse los valores) y luego guardar el archivo

---
agregar si hubiera un archivo requirements
RUN pip install --no-cache-dir -r requirements.txt

In [ ]:
# definir path folders
folder_backend = f"gs://{BUCKET_MLFLOW_CLOUDRUN}" + "/mlflow_backend"
folder_artifacts = f"gs://{BUCKET_MLFLOW_CLOUDRUN}" + "/mlflow_artifacts"


print('backend: ', folder_backend)
print('artifacts: ', folder_artifacts)

In [ ]:
# crear string que representa al dockerfile - fijando artifacts y backend

string_dockerfile = f'''
FROM python:3.8-slim-buster

# Copiar los archivos del proyecto de MLflow a la imagen de Docker
COPY . /app
WORKDIR /app

# Instala las dependencias necesarias
RUN pip install google-cloud-storage mlflow==2.2.1

# Copia el archivo de credenciales a la imagen de Docker
COPY {path_credentials} /app/{path_credentials}

# Establece la variable de entorno GOOGLE_APPLICATION_CREDENTIALS
ENV GOOGLE_APPLICATION_CREDENTIALS=/app/{path_credentials}

# Establecer el Bucket para la conexión con mlflow
ENV MLFLOW_MODEL_REGISTRY_URI = gs://{BUCKET_MLFLOW_CLOUDRUN}

# Inicia el servidor de MLflow
CMD mlflow server --backend-store-uri {folder_backend} --default-artifact-root {folder_artifacts} --host 0.0.0.0 --port 8080
'''

In [ ]:
# guardar dockerfile
with open('Dockerfile', 'w') as file:
    file.write(string_dockerfile)

### 5. Crear repo en artifact Registry
- Para crear un servicio de cloud se hace a partir de una imagen (dockerfile creado en el paso anterior) y dicha imagen debe ser almacenada en un repo de "Artifact Registry"

- Se procede a crear el repo

In [ ]:
# crear repo
! gcloud artifacts repositories create $NAME_REPO \
--repository-format $FORMAT_REPO \
--location $REGION \
--description "$DESCRIPTION_REPO" \
--async

### 6. Configurar una compilación de Docker
Es necesario crear un yaml con la configuración para compilar la imagen docker en Artifact Registry

-> El siguiente código está parametrizado y no es necesario tocar para ninguna implementación de cloud run

In [ ]:
# crear diccionario python con el contenido del yaml cloudbuild genérico
dict_python_yaml_cloudbuild = {'steps': [{'name': 'gcr.io/cloud-builders/docker',
   'args': ['build', '-t', '${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}', '.']}],
 'images': ['${_LOCATION}-docker.pkg.dev/$PROJECT_ID/${_REPOSITORY}/${_IMAGE}']}


# guardar diccionario en formato yaml
with open(r'cloudbuild.yaml', 'w') as file:
    documents = yaml.dump(dict_python_yaml_cloudbuild, file)

### 7. Contenerizar (imagen docker) utilizando cloud build y subirlas a artifact registry

In [ ]:
! gcloud builds submit \
    --config=cloudbuild.yaml \
    --substitutions=_LOCATION="$REGION",_REPOSITORY="$NAME_REPO",_IMAGE="$NAME_IMAGE" .

### 8. Deploy de la imagen del contenedor de artifact registry en cloud run


**IMPORTANTE: POR PROBLEMAS DE PERMISOS, EL CLOUD RUN ESTÁ CONFIGURADO PARA QUE CUALQUIERA CON EL LINK PUEDA ACCEDER**

In [ ]:
! gcloud run deploy $NAME_CLOUD_RUN \
    --image $REGION-docker.pkg.dev/$PROJECT_ID/$NAME_REPO/$NAME_IMAGE \
    --region $REGION \
    --allow-unauthenticated